In [15]:
import spacy
spacy.load('en')
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

from spacy.lang.en import English
%matplotlib inline

In [6]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [9]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/apcloud44/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [11]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/apcloud44/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [16]:
conn = sqlite3.connect("../../data/twitter.db")
df_bitcoin = pd.read_sql_query('select TweetText from bitcoin where DateTime > "2018-03-03" and DateTime < "2018-05-03" and Language="en"', conn)
#df_eth = pd.read_sql_query('select TweetText from eth where DateTime > "2018-03-03" and DateTime < "2018-05-03" ', conn)
#df_facebook = pd.read_sql_query('select TweetText from facebook where DateTime > "2018-03-03" and DateTime < "2018-05-03" ', conn)
df_all = df_bitcoin #pd.concat([df_bitcoin, df_eth, df_facebook], sort=False)

In [30]:
text_data = []
for row in df_all.iloc[:10,0]:
    tokens = prepare_text_for_lda(line)
    print(tokens)
    text_data.append(tokens)

['1024384316128808960,1024384316128808960,864932553697964032,dailycoinfeed,0,2018', '16:00,top', 'cryptocurrencies', 'alert', '23:00:27', 'bitcoin', '7;785.645', 'ethereum', '436.044', '0.436', 'bitcoincash', '781.860', '7.325', 'ripplenews', 'trading', 'steem,0,0,0,,#cryptocurrencies', 'bitcoin', 'ethereum', 'bitcoincash', 'ripplenews', 'trading', 'steem,/dailycoinfeed', 'status/1024384316128808960,,0,0,0']
['1024384316128808960,1024384316128808960,864932553697964032,dailycoinfeed,0,2018', '16:00,top', 'cryptocurrencies', 'alert', '23:00:27', 'bitcoin', '7;785.645', 'ethereum', '436.044', '0.436', 'bitcoincash', '781.860', '7.325', 'ripplenews', 'trading', 'steem,0,0,0,,#cryptocurrencies', 'bitcoin', 'ethereum', 'bitcoincash', 'ripplenews', 'trading', 'steem,/dailycoinfeed', 'status/1024384316128808960,,0,0,0']
['1024384316128808960,1024384316128808960,864932553697964032,dailycoinfeed,0,2018', '16:00,top', 'cryptocurrencies', 'alert', '23:00:27', 'bitcoin', '7;785.645', 'ethereum', '4

In [31]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [36]:
import pickle
pickle.dump(corpus, open('../../data/corpus.pkl', 'wb'))
dictionary.save('../../data/dictionary.gensim.all')

In [37]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('../../data/model_lsi.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.056*"ripplenews" + 0.056*"bitcoincash" + 0.056*"bitcoin" + 0.056*"trading"')
(1, '0.086*"bitcoincash" + 0.086*"trading" + 0.086*"ripplenews" + 0.086*"ethereum"')
(2, '0.057*"bitcoin" + 0.057*"ethereum" + 0.057*"ripplenews" + 0.057*"trading"')
(3, '0.056*"bitcoin" + 0.056*"ethereum" + 0.056*"ripplenews" + 0.056*"trading"')
(4, '0.056*"ripplenews" + 0.056*"bitcoin" + 0.056*"ethereum" + 0.056*"trading"')


In [38]:
dictionary = gensim.corpora.Dictionary.load('../../data/dictionary.gensim.all')
corpus = pickle.load(open('../../data/corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('../../data/model_lsi.gensim')

In [39]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/home/apcloud44/miniconda3/envs/p3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
